In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, Model
import efficientnet.tfkeras as efn
import tensorflow as tf
from tensorflow.keras.callbacks import CSVLogger


size = 100
# Load and preprocess the data
(x_train, y_train), (x_test, y_test) = cifar100.load_data(
    label_mode="fine")



x_train = tf.image.resize(x_train, (size, size))
x_test = tf.image.resize(x_test, (size, size))

#x_train = x_train.astype('float32') / 255.0
#x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

# Create data augmentation generator
data_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.15
)
data_gen.fit(x_train)

# Load EfficientNet-B0 without pre-trained weights
efficientnet = efn.EfficientNetB3(weights='imagenet', include_top=False, input_shape=(size, size, 3))

# Add a custom classification head
x = layers.GlobalAveragePooling2D()(efficientnet.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(100, activation='softmax')(x)

# Create the combined model
combined_model = Model(inputs=efficientnet.input, outputs=predictions)

opt = Adam(learning_rate=0.001)
combined_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Set up the callbacks
os.makedirs("checkpoints", exist_ok=True)
os.makedirs("plots", exist_ok=True)

checkpoint = ModelCheckpoint("checkpoints/weights.{epoch:02d}.hdf5", save_freq=20)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=5, verbose=1, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='loss', patience=10, verbose=1)
csv_logger = CSVLogger("plots/training_log.csv")

# Train the model
history = combined_model.fit(
    data_gen.flow(x_train, y_train, batch_size=64),
    epochs=100,
    verbose=1,
    steps_per_epoch=len(x_train) // 64,
    callbacks=[checkpoint, reduce_lr, early_stopping, csv_logger]
)

# Save the final model
combined_model.save("efficientnetfinetunedB3.h5")

# Plot the learning curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(history.history['loss'], label='loss')
ax1.plot(history.history['accuracy'], label='accuracy')
ax1.set_title('efficientnet Loss')
ax1.legend()

ax2.plot(history.history['accuracy'], label='train')
ax2.plot(history.history['accuracy'], label='validation')
ax2.set_title('efficientnet Accuracy')
ax2.legend()

plt.savefig("plots/learning_curves.png")

test_loss, test_accuracy = combined_model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

# Plot the learning curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(history.history['loss'], label='loss')
ax1.plot(history.history['accuracy'], label='accuracy')
ax1.plot(history.history['lr'], label='learning rate')
ax1.set_title('efficientnet')
ax1.legend()


plt.savefig("plots/learning_curves.png")

test_loss, test_accuracy = combined_model.evaluate(x_test, y_test)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")